## Generating binary categories for training
**Steps**

- Setup categoricals for each segment based on author consensus
- Assign a binary value to each category of segment, in corresponding category columns (one-hot encoding)

In [42]:
import pandas as pd
import numpy as np
import nltk

In [43]:
# from sqlalchemy import create_engine
# from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [44]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [45]:
#Connect to db
dbname = 'beforeiagree_db'
username = 'peterostendorp'

#Create engine
con = psycopg2.connect(database = dbname, user = username)

In [98]:
#Get annotations df
sql = """
SELECT * FROM annotations
WHERE "Policy UID" IN
(SELECT "Policy UID" FROM sites
WHERE "In 115 Set?" = TRUE);
"""
annotations = pd.read_sql_query(sql,con)

In [100]:
#Get sites df
sql = """
SELECT * FROM sites
WHERE "In 115 Set?" = TRUE;
"""
sites = pd.read_sql_query(sql,con)

In [101]:
#Get segments df
sql = """
SELECT * FROM segments
WHERE "Policy UID" IN
(SELECT "Policy UID" FROM sites
WHERE "In 115 Set?" = TRUE)
"""

segments = pd.read_sql_query(sql,con)

In [102]:
print(annotations.shape)
annotations.head()

(23194, 9)


,Policy UID,annotation_id,batch_id,annotator_id,segment_id,category_name,attributes_value_pairs,date,policy_url
0,1017,20137,test_category_labeling_highlight_fordham_aaaaa,121,0,Other,"{""Other Type"": {""selectedText"": ""Sci-News.com ...",NaT,http://www.sci-news.com/privacy-policy.html
1,1017,20324,test_category_labeling_highlight_fordham_aaaaa,121,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",NaT,http://www.sci-news.com/privacy-policy.html
2,1017,20325,test_category_labeling_highlight_fordham_aaaaa,121,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",NaT,http://www.sci-news.com/privacy-policy.html
3,1017,20326,test_category_labeling_highlight_fordham_aaaaa,121,2,Data Retention,"{""Personal Information Type"": {""selectedText"":...",NaT,http://www.sci-news.com/privacy-policy.html
4,1017,20327,test_category_labeling_highlight_fordham_aaaaa,121,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele...",NaT,http://www.sci-news.com/privacy-policy.html


In [103]:
print(segments.shape)
segments.head()

(6469, 3)


,Policy UID,segment_id,segments
0,20,0,<strong> Privacy Policy </strong> <br> <br> <s...
1,20,1,This privacy policy does not apply to Sites ma...
2,20,2,"By visiting our Sites, you are accepting the p..."
3,20,3,<strong> What Information Is Collected? </stro...
4,20,4,<strong> Personally Identifiable Information <...


In [9]:
sites.head()

,Policy UID,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sector,Policy URL,Policy collection date,Policy last updated date,policy_text
0,20,1,theatlantic.com,The Atlantic,2016-02-08,True,"Alexa Rank: 975 (Global), 289 (US)",Arts,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01,<strong> Privacy Policy </strong> <br> <br> <s...
1,21,2,imdb.com,IMDb,2016-02-08,True,"Alexa Rank: 49 (Global), 27 (US)",Arts,imdb.com/privacy,2015-07-02,2014-12-05,"IMDb Privacy Notice <br> <br>|||Last Updated, ..."
2,26,3,nytimes.com,New York Times,2016-02-08,True,"Alexa Rank: 101 (Global), 22 (US)",Arts,nytimes.com/privacy,2015-07-08,2015-06-10,<strong> Privacy Policy </strong> <br> <br> La...
3,32,4,theverge.com,The Verge,2016-02-08,True,"Alexa Rank: 525 (Global), 230 (US)",Home,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...
4,32,1000,voxmedia.com,Vox Media,2015-02-14,False,"Alexa Rank: 40248 (Global), 18551 (US)",None,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...


In [104]:
sql = """
SELECT segments."Policy UID", segments.segment_id, segments.segments, 
annotations.category_name FROM segments
INNER JOIN annotations ON segments."Policy UID"=annotations."Policy UID"
AND segments.segment_id=annotations.segment_id
WHERE segments."Policy UID" IN
(SELECT "Policy UID" FROM sites
WHERE "In 115 Set?" = TRUE);
"""
join_segments = pd.read_sql_query(sql,con)
join_segments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40133 entries, 0 to 40132
Data columns (total 4 columns):
Policy UID       40133 non-null int64
segment_id       40133 non-null int64
segments         40133 non-null object
category_name    40133 non-null object
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


In [106]:
#Set up table in which categories are assigned to each unique document segment. Those that aren't
#tagged are labeled as "None".
#Join category name to segments and determine the mode of each category, concat back onto original segments
tmp = pd.merge(segments,annotations,on=['Policy UID','segment_id'],how='outer')
tmp['category_name'].fillna(value='None')
tmp2 = pd.DataFrame(tmp.groupby(by=['Policy UID','segment_id']).count())
print(tmp2.shape)
tmp2.head(100)#['category_name'].agg(lambda x: x.value_counts().index[0]))

(3792, 8)


segments  annotation_id  batch_id  annotator_id  \
Policy UID segment_id                                                    
20         0                  3              3         3             3   
           1                  3              3         3             3   
           2                  7              7         7             7   
           3                 13             13        13            13   
           4                  5              5         5             5   
           5                 16             16        16            16   
           6                 10             10        10            10   
           7                  7              7         7             7   
           8                  4              4         4             4   
           9                  7              7         7             7   
           10                 9              9         9             9   
           11                15             15        15            15   
           12                 3              3         3             3   
           13                20             20        20            20   
           14                 3              3         3             3   
           15                 2              2         2             2   
           16                12             12        12            12   
           17                 6              6         6             6   
           18                 3              3         3             3   
           19                 5              5         5             5   
           20                 5              5         5             5   
           21                 4              4         4             4   
           22                 4              4         4             4   
           23                 7              7         7             7   
           24                 3              3         3             3   
           25                 3              3         3             3   
           26                 3              3         3             3   
           27                 3              3         3             3   
           28                 3              3         3             3   
           29                 6              6         6             6   
...                         ...            ...       ...           ...   
21         34                 2              2         2             2   
           35                 6              6         6             6   
26         0                  3              3         3             3   
           1                  3              3         3             3   
           2                  3              3         3             3   
           3                  3              3         3             3   
           4                  3              3         3             3   
           5                  3              3         3             3   
           6                  3              3         3             3   
           7                  3              3         3             3   
           8                  3              3         3             3   
           9                 12             12        12            12   
           10                 3              3         3             3   
           11                 9              9         9             9   
           12                 4              4         4             4   
           13                 9              9         9             9   
           14                 3              3         3             3   
           15                 9              9         9             9   
           16                10             10        10            10   
           17                 4              4         4             4   
           18                 3              3         3             3   
           19              

In [85]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40133 entries, 0 to 40132
Data columns (total 10 columns):
Policy UID                40133 non-null int64
segment_id                40133 non-null int64
segments                  40133 non-null object
annotation_id             40133 non-null int64
batch_id                  40133 non-null object
annotator_id              40133 non-null int64
category_name             40133 non-null object
attributes_value_pairs    40133 non-null object
date                      36399 non-null datetime64[ns]
policy_url                40133 non-null object
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 3.4+ MB


In [60]:
categories = list(tmp2['category_name'].unique())
cols = {'Other': 'other',
        'Policy Change': 'policy_change',
        'First Party Collection/Use': 'first_party_collection_use',
        'Third Party Sharing/Collection': 'third_party_sharing_collection',
        'Do Not Track': 'do_not_track',
        'User Choice/Control': 'user_choice_control',
        'International and Specific Audiences': 'international_specific_audiences',
        'Data Security': 'data_security',
        'Data Retention': 'data_retention',
        'User Access, Edit and Deletion': 'user_access_edit_deletion'}

## Segment-level categorization
Loop through categories and generate one-hot encoding of each category and stick them into new columns with the corresponding names in cols

In [61]:
#Loop through the categories and generate a set of new columns with names in cols
binary_categories = pd.DataFrame()

for category in categories:
    one_hot = lambda s: 1 if s.startswith(category) else 0
    binary_categories[cols[category]] = tmp2['category_name'].apply(one_hot)

In [65]:
binary_categories.head()

other  policy_change  first_party_collection_use  \
Policy UID segment_id                                                     
20         0               1              0                           0   
           1               1              0                           0   
           2               0              1                           0   
           3               0              0                           1   
           4               0              0                           1   

                       third_party_sharing_collection  do_not_track  \
Policy UID segment_id                                                 
20         0                                        0             0   
           1                                        0             0   
           2                                        0             0   
           3                                        0             0   
           4                                        0             0   

                       user_choice_control  international_specific_audiences  \
Policy UID segment_id                                                          
20         0                             0                                 0   
           1                             0                                 0   
           2                             0                                 0   
           3                             0                                 0   
           4                             0                                 0   

                       data_security  data_retention  \
Policy UID segment_id                                  
20         0                       0               0   
           1                       0               0   
           2                       0               0   
           3                       0               0   
           4                       0               0   

                       user_access_edit_deletion  
Policy UID segment_id                             
20         0                                   0  
           1                                   0  
           2                                   0  
           3                                   0  
           4                                   0

In [27]:
#Create engine for persisting
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [28]:
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://peterostendorp@localhost/beforeiagree_db


In [29]:
binary_categories.to_sql('binary_segment_categories',engine, if_exists='replace')

## Policy-level categorization

In [39]:
binary_categories_policy = binary_categories.reset_index(level=[0,1]).groupby('Policy UID').sum().drop('segment_id',axis=1)
binary_categories_policy = binary_categories_policy.applymap(lambda x: 1 if x > 0 else 0)
binary_categories_policy.head()

,other,policy_change,first_party_collection_use,third_party_sharing_collection,do_not_track,user_choice_control,international_specific_audiences,data_security,data_retention,user_access_edit_deletion
Policy UID,,,,,,,,,,
20,1,1,1,1,1,1,1,1,1,1
21,1,1,1,1,0,1,1,1,0,1
26,1,1,1,1,0,1,1,1,0,1
32,1,1,1,1,0,1,1,1,1,0
33,1,1,1,1,0,1,1,1,1,1


In [40]:
binary_categories_policy.sum()

other                               112
policy_change                        83
first_party_collection_use          112
third_party_sharing_collection      110
do_not_track                         22
user_choice_control                  98
international_specific_audiences     85
data_security                        94
data_retention                       25
user_access_edit_deletion            73
dtype: int64

Most policies contain info on:
- Other
- First and third party data collection
- user choice control
- data security

Less frequently mentioned:
- data retention
- user access edit deletion
- do not track

In [41]:
binary_categories_policy.to_sql('binary_policy_categories',engine)